In [1]:
import pyscal.core as pc
import pyscal.crystal_structures as pcs
import os
import numpy as np
from tqdm import tqdm

In [2]:
from pymolsim import Sim, Thermostat, Barostat
from pymolsim.potential import LJ

In [3]:
sys = pc.System()
sys.read_inputfile("conf.dump", customkeys=["mass", "vx", "vy", "vz"])
atoms = sys.atoms
box = sys.box

In [4]:
sim = Sim(atoms, box)

In [5]:
sim.beta = 0.5
sim.pressure = 0
sim.dt = 0.0025
lj = LJ(1.0, 1.0)
sim.potential = lj
sim.start()
sim.forces()

/home/sarath/miniconda3/envs/lammps/lib/python3.7/site-packages/pymolsim-0.0.1-py3.7.egg/pymolsim/potential.py:34: RuntimeWarning: divide by zero encountered in true_divide
/home/sarath/miniconda3/envs/lammps/lib/python3.7/site-packages/pymolsim-0.0.1-py3.7.egg/pymolsim/potential.py:38: RuntimeWarning: invalid value encountered in true_divide


In [10]:
%%timeit
sim.forces()

17.2 ms ± 757 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit
sim.forces()

3.68 s ± 55.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
sim.px/sim.nparticles/sim.mass

array([-0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00597337,
       -0.00597337, -0.00597337, -0.00597337, -0.00597337, -0.00

In [9]:
sim.forces()
sim.remap()

In [10]:
sim.dump(0)

In [11]:
for i in tqdm(range(1, 10)):
    sim.md_verlet()
    sim.remap()
    sim.dump(i)

100%|██████████| 9/9 [00:34<00:00,  3.84s/it]
